In [8]:
from imdbparser import IMDb
imdb = IMDb()
movie = imdb.get_movie(1733125)
movie.fetch()
print(movie.__dict__)

{'imdb_id': '1733125', 'imdb': <imdbparser.imdb.IMDb object at 0x10909c0f0>, 'trees': [<Element html at 0x109c03a98>, <Element html at 0x10695b8b8>], 'alternative_titles': ['Dungeons & Dragons 3', 'Dungeons & Dragons 3: The Book of Vile Darkness', 'Lochy i smoki III', 'Dungeons & Dragons 3: Das Buch der Dunklen Schatten', 'Donjons & Dragons 3: Le Livre des Ténèbres'], 'actors': [<Person fetched=False imdb_id='0397928' name='Anthony Howell'>, <Person fetched=False imdb_id='0535582' name='Dominic Mafham'>, <Person fetched=False imdb_id='2304132' name='Jack Derges'>, <Person fetched=False imdb_id='3845763' name='Beau Brasseaux'>, <Person fetched=False imdb_id='0900770' name='Kaloian Vodenicharov'>, <Person fetched=False imdb_id='2409667' name='Charlotte Hunter'>, <Person fetched=False imdb_id='2806024' name='Eleanor Gecks'>, <Person fetched=False imdb_id='4742381' name='Austin Naulty'>, <Person fetched=False imdb_id='4476832' name='Sebastian James'>, <Person fetched=False imdb_id='1284927

In [ ]:
from imdbparser import IMDb
imdb = IMDb()
movie_dict_list = []
with open('imdb_id_list.txt','r') as fp:
    for line in fp: 
        movie = imdb.get_movie(line[2:9])
        movie.fetch()
        movie_dict_list.append(movie.__dict__)
        print("success")
print(len(movie_dict_list))

In [14]:
print(len(movie_dict_list))

1447


In [16]:
alternate = movie_dict_list
print(len(alternate))

1447


In [20]:
import json
with open("movie_dict.json", 'w') as fout:
    for row in alternate:
        if 'imdb' in row:
            del row['imdb']
        if 'trees' in row:
            del row['trees']
        if 'actors' in row:
            del row['actors']
        if 'directors' in row:
            del row['directors']
        if 'writers' in row:
            del row['writers']
        if 'rating' in row:
            del row['rating']
        print(json.dumps(row), file=fout)        

In [40]:
import urllib.request, urllib.parse, urllib.error
import json
# my_key = '71b7030c'
# my_key = 'a873cc3'
# my_key = '154ff3c'
# my_key = '1cd4cefc'
# my_key = 'e84dff6c'
# my_key = 'a53b4479'
my_key = '50c60ef1'
# my_key = '197cc3dd'
# my_key = 'f53a1526'
serviceurl = 'http://www.omdbapi.com/?'
apikey = '&apikey='+ my_key
movie_dict_list = []
with open('imdb_id_list.txt','r') as fp:
    for i, line in enumerate(fp): 
        if i >= 14799:
            movie_dict = {'imdb_id': line[:9]}
            movie_id = 'i='+ line[:9]
            try:
                url = serviceurl + movie_id + apikey + '&plot=full'
                uh = urllib.request.urlopen(url)
                data = uh.read()
                json_data=json.loads(data)

                if json_data['Response']=='True':
                    movie_dict.update(json_data)
                    movie_dict_list.append(movie_dict)
                    print('line number: ' + str(i))
                else:
                    print("Error encountered: ",json_data['Error'])
                    break

            except urllib.error.URLError as e:
                print(f"ERROR: {e.reason}")
                break

line number: 14799
line number: 14800
line number: 14801
line number: 14802
line number: 14803
line number: 14804
line number: 14805
line number: 14806
line number: 14807
line number: 14808
line number: 14809
line number: 14810
line number: 14811
line number: 14812
line number: 14813
line number: 14814
line number: 14815
line number: 14816
line number: 14817
line number: 14818
line number: 14819
line number: 14820
line number: 14821
line number: 14822
line number: 14823
line number: 14824
line number: 14825
line number: 14826
line number: 14827


In [41]:
print(len(movie_dict_list))

29


In [25]:
alternate = movie_dict_list
print(len(alternate)) #6171 -- 6984

814


In [3]:
with open("movie_dict_3.json", 'w') as fout:
    for row in movie_dict_list:
        print(json.dumps(row), file=fout)

In [42]:
with open("movie_dict_3.json", 'a') as fout:
    for row in movie_dict_list:
        print(json.dumps(row), file=fout)

In [1]:
import json
movie_list = []
with open("movie_dict_3.json", "r")as fin:
    for line in fin:
        movie_json = json.loads(line)
        fout = movie_json['imdb_id'] + ".txt"
        movie_list.append(movie_json)
#         if len(movie_list) > 9:
#             break

tt0057603.txt
tt1733125.txt
tt0033045.txt
tt0113862.txt
tt0086250.txt
tt1315981.txt
tt0249380.txt
tt0408790.txt
tt0021079.txt
tt1615065.txt


# Keyword crawl from tmdb

In [15]:
import tmdbsimple as tmdb
import json
import time

tmdb.API_KEY = 'e14e5a01170cc653d584cedf1a036fef'

movie_list = []
not_matched = []
not_matched_tmdb = []
with open("movie_dict_3.json", "r")as fin:
    for i, line in enumerate(fin):
        if i%20 == 0:
            time.sleep(20)
        movie_json = json.loads(line)
        find = tmdb.Find(movie_json['imdb_id'])
        response = find.info(external_source='imdb_id')
        if find.movie_results:
            find_result = find.movie_results[0]
            if find_result['title'] == movie_json['Title']:
                movie = tmdb.Movies(find_result['id'])
                key_response = movie.keywords()
                if key_response:
                    keywords_list = []
                    for item in key_response['keywords']:
                        keywords_list.append(item['name'])
                    movie_json['keywords'] = keywords_list
                    movie_list.append(movie_json)
                else:
                    print("no keyword response")
            else:
                print("titles don't match")
                print(movie_json['Title'] + "tmdb_title: " + find_result['title'])
                not_matched.append(movie_json['imdb_id'])
                not_matched_tmdb.append(find_result['id'])
        else:
            print("no find response")

titles don't match
Rape Metmdb_title: Baise-moi
no find response
titles don't match
Scream of Feartmdb_title: Taste of Fear
titles don't match
Rivalstmdb_title: The Stalking of Laurie Show
titles don't match
Die Hard with a Vengeancetmdb_title: Die Hard: With a Vengeance
no find response
no find response
titles don't match
The Road Warriortmdb_title: Mad Max 2
titles don't match
Ashoka the Greattmdb_title: Aśoka
no find response
no find response
no find response
no find response
no find response
titles don't match
9 to 5tmdb_title: Nine to Five
titles don't match
Tristan + Isoldetmdb_title: Tristan & Isolde
titles don't match
Star Wars: Episode V - The Empire Strikes Backtmdb_title: The Empire Strikes Back
titles don't match
Il Postinotmdb_title: The Postman
no find response
no find response
titles don't match
The Haunting of Juliatmdb_title: Full Circle
titles don't match
Embrace the Darkness 3tmdb_title: Embrace the Darkness III
titles don't match
Jackie Chan's Who Am I?tmdb_title: W

no find response
titles don't match
Cabin Fever 3: Patient Zerotmdb_title: Cabin Fever: Patient Zero
titles don't match
To Save a Lifetmdb_title: To Save A Life
titles don't match
Undisputed 2: Last Man Standingtmdb_title: Undisputed II: Last Man Standing
titles don't match
Supercoptmdb_title: Police Story 3: Supercop
titles don't match
Spaceflight IC-1: An Adventure in Spacetmdb_title: Spaceflight IC-1
titles don't match
StageFrighttmdb_title: StageFright: Aquarius
titles don't match
El Crimen Perfecto (The Perfect Crime)tmdb_title: Ferpect Crime
no find response
titles don't match
Star Wars: Episode VI - Return of the Jeditmdb_title: Return of the Jedi
titles don't match
The Russians Are Coming! The Russians Are Coming!tmdb_title: The Russians Are Coming, The Russians Are Coming
titles don't match
A Friend to Die Fortmdb_title: Death of a Cheerleader
titles don't match
The Hand That Rocks the Cradletmdb_title: The Hand that Rocks the Cradle
no find response
titles don't match
Poison 

titles don't match
Fantastic 4: Rise of the Silver Surfertmdb_title: Fantastic Four: Rise of the Silver Surfer
titles don't match
The Butcher of Praguetmdb_title: Lidice
titles don't match
Michael Jackson: Thrillertmdb_title: Michael Jackson's Thriller
no find response
no find response
no find response
titles don't match
Battle Los Angelestmdb_title: Battle: Los Angeles
titles don't match
Mousehunttmdb_title: MouseHunt
titles don't match
As Long As I Livetmdb_title: Jab Tak Hai Jaan
no find response
titles don't match
Ever After: A Cinderella Storytmdb_title: EverAfter
titles don't match
Sesh Ankatmdb_title: The Last Act
titles don't match
Dragonball: Evolutiontmdb_title: Dragonball Evolution
no find response
titles don't match
Raw Meattmdb_title: Death Line
no find response
titles don't match
Murderers Among Ustmdb_title: The Murderers Are Among Us
titles don't match
Jeepers Creepers IItmdb_title: Jeepers Creepers 2
titles don't match
Underworld Awakeningtmdb_title: Underworld: Awaken

titles don't match
Young@Hearttmdb_title: Young @ Heart
titles don't match
Warcraft: The Beginningtmdb_title: Warcraft
titles don't match
Foreign Landtmdb_title: Pardes
titles don't match
¡Vampiros en La Habana!tmdb_title: Vampires in Havana
no find response
no find response
no find response
titles don't match
The Tsunami Warriortmdb_title: Queens of Langkasuka
titles don't match
I'm Not Theretmdb_title: I'm Not There.
titles don't match
Ma Vie en Rosetmdb_title: My Life in Pink
no find response
no find response
titles don't match
Sherlock Jr.tmdb_title: Sherlock, Jr.
titles don't match
Searchtmdb_title: Talaash
no find response
titles don't match
This Boy's Lifetmdb_title: This Boy’s Life
titles don't match
5 Dolls for an August Moontmdb_title: Five Dolls for an August Moon
no find response
titles don't match
The Falcon and the Co-edstmdb_title: The Falcon and the Co-Eds
no find response
titles don't match
Confidential Reporttmdb_title: Mr. Arkadin
titles don't match
4.3.2.1.tmdb_titl

no find response
no find response
titles don't match
Eye See Youtmdb_title: D-Tox
titles don't match
Les visiteurstmdb_title: The Visitors
titles don't match
The Web of the Witchtmdb_title: Makdee
titles don't match
O Heart, It Is Difficulttmdb_title: Ae Dil Hai Mushkil
no find response
titles don't match
Les Visiteurs du Soirtmdb_title: The Evening Visitors
no find response
no find response
titles don't match
Swordsman IItmdb_title: The Legend of the Swordsman
no find response
titles don't match
Gojô reisenki: Gojoetmdb_title: Gojoe: Spirit War Chronicle
no find response
no find response
no find response
no find response
no find response
titles don't match
Closuretmdb_title: Nine Inch Nails: Closure
titles don't match
Mamatmdb_title: Mommy
no find response
titles don't match
Class of 1999 II: The Substitutetmdb_title: Class of 1999 II - The Substitute
titles don't match
The Snowtown Murderstmdb_title: Snowtown
no find response
no find response
titles don't match
The Burma Conspiracytm

...

no find response
titles don't match
Scooby-Doo and the Reluctant Werewolftmdb_title: Scooby-Doo! and the Reluctant Werewolf
titles don't match
Navajerostmdb_title: Candy Razors
titles don't match
The Night Before Christmastmdb_title: Troublemakers
no find response
titles don't match
Shall We Dancetmdb_title: Shall We Dance?
titles don't match
The Uninvitedtmdb_title: Uninvited
no find response
titles don't match
The Tom and Jerry Movietmdb_title: Tom and Jerry: The Movie
no find response
no find response
titles don't match
Matando Cabostmdb_title: Killing Cabos
titles don't match
Carry on Cleotmdb_title: Carry On Cleo
no find response
no find response
titles don't match
Maladolescenzatmdb_title: Playing with Love
titles don't match
Carry on Girlstmdb_title: Carry On Girls
titles don't match
13 Hours by Airtmdb_title: Thirteen Hours by Air
titles don't match
The Great Train Robberytmdb_title: The First Great Train Robbery
no find response
no find response
no find response
no find respon

titles don't match
Mobile Suit Gundam 0083: Stardust Memory: The Mayfly of Spacetmdb_title: Mobile Suit Gundam 0083: Stardust Memory - The Mayfly of Space
titles don't match
My Stepmother Is an Alientmdb_title: My Stepmother is an Alien
no find response
no find response
no find response
no find response
no find response
no find response
no find response
titles don't match
Ashes of Paradisetmdb_title: Cenizas del Paraiso
no find response
no find response
no find response
titles don't match
Body & Soultmdb_title: The Sex Substitute
titles don't match
Spy Kids 3: Game Overtmdb_title: Spy Kids 3-D: Game Over
no find response
no find response
titles don't match
Rory O'Shea Was Heretmdb_title: Inside I'm Dancing
no find response
no find response
titles don't match
Monsters vs Aliens: Mutant Pumpkins from Outer Spacetmdb_title: Mutant Pumpkins from Outer Space
no find response
no find response
titles don't match
Naked Killer 2tmdb_title: Raped by an Angel
titles don't match
R.L. Stine's Monst

no find response
titles don't match
Khoobsurattmdb_title: Beautiful
no find response
no find response
no find response
no find response
titles don't match
Shake, Rattle & Roll 9tmdb_title: Shake, Rattle and Roll 9
no find response
titles don't match
Hamlettmdb_title: Hamlet, Prince of Denmark
no find response
no find response
no find response
no find response
no find response
titles don't match
Yu Yu Hakusho: Fight for the Netherworldtmdb_title: Yu Yu Hakusho: The Movie - Poltergeist Report
titles don't match
Caro diariotmdb_title: Dear Diary
titles don't match
I Am the Cheesetmdb_title: I Am The Cheese
titles don't match
Five Million Years to Earthtmdb_title: Quatermass and the Pit
no find response
titles don't match
Hot Wheels Highway 35 World Racetmdb_title: Hot Wheels: World Race
titles don't match
The Double Life of Veroniquetmdb_title: The Double Life of Véronique
no find response
titles don't match
Poojatmdb_title: Poojai
titles don't match
The Mystery of Edwin Droodtmdb_title: 

no find response
titles don't match
Seeta Aur Geetatmdb_title: Seeta and Geeta
titles don't match
Gor IItmdb_title: Outlaw of Gor
no find response
titles don't match
Larger Than Lifetmdb_title: Larger than Life
no find response
titles don't match
Victim of Lovetmdb_title: Victim Of Love
no find response
no find response
titles don't match
30,000 Leagues Under the Seatmdb_title: 30,000 Leagues Under The Sea
no find response
no find response
no find response
no find response
titles don't match
The Dawn Expresstmdb_title: Dawn Express
titles don't match
Charlie Brown's All Stars!tmdb_title: Charlie Brown's All-Stars
titles don't match
Death in the Airtmdb_title: Pilot X
no find response
no find response
titles don't match
Dark Touristtmdb_title: The Grief Tourist
no find response
titles don't match
Murder Is Easytmdb_title: Murder is Easy
no find response
titles don't match
Travelers and Magicianstmdb_title: Travellers and Magicians
no find response
no find response
no find response
no fi

no find response
no find response
titles don't match
Only the Bravetmdb_title: Only The Brave
no find response
titles don't match
The Hypnotizedtmdb_title: Hypnotized
titles don't match
Schramm: Into the Mind of a Serial Killertmdb_title: Schramm
titles don't match
The Demon Barber of Fleet Streettmdb_title: Sweeney Todd: The Demon Barber of Fleet Street
titles don't match
Hansel and Greteltmdb_title: Engelbert Humperdinck: Hansel and Gretel
no find response
no find response
titles don't match
ffolkestmdb_title: North Sea Hijack
no find response
no find response
no find response
no find response
no find response
no find response
no find response
no find response
no find response
no find response
no find response
titles don't match
Ride Him, Bosko!tmdb_title: Ride Him, Bosko
titles don't match
Power Rangerstmdb_title: Power/Rangers
no find response
no find response
no find response
titles don't match
Highlander II: The Quickeningtmdb_title: Highlander 2: The Quickening
titles don't matc

In [16]:
len(movie_list)

12666

In [17]:
len(not_matched)

1051

In [25]:
not_matched_tmdb[0]

3134

In [32]:
not_matched[0]

'tt0249380'

In [35]:
set_not_matched = set(not_matched)
movie_list_keyword = []

count = 0
with open("movie_dict_3.json", "r")as fin:
    for line in fin:
        movie_json = json.loads(line)
        if movie_json['imdb_id'] in set_not_matched:
            count += 1
            if count%20 == 0:
                time.sleep(20)
                
            find = tmdb.Find(movie_json['imdb_id'])
            response = find.info(external_source='imdb_id')
            if find.movie_results:
                find_result = find.movie_results[0]
                movie = tmdb.Movies(find_result['id'])
                try:
                    key_response = movie.keywords()
                    if key_response:
                        keywords_list = []
                        for item in key_response['keywords']:
                            keywords_list.append(item['name'])
                        movie_json['keywords'] = keywords_list
                        movie_list_keyword.append(movie_json)
                    else:
                        print("no keyword response")
                except:
                    print("error with tmdb id: " + str(find_result['id']))
            else:
                print("no find response")

error with tmdb id: 160266


In [36]:
len(movie_list_keyword)

1050

In [38]:
movie_list_keyword[-2]

{'Actors': 'Thomas Schmidt, Johannes Maus, Friedrich Richter, Trude Hesterberg',
 'Awards': 'N/A',
 'BoxOffice': 'N/A',
 'Country': 'East Germany',
 'DVD': 'N/A',
 'Director': 'Wolfgang Staudte',
 'Genre': 'Family, Fantasy',
 'Language': 'German',
 'Metascore': 'N/A',
 'Plot': 'An old man living in an oriental city tells the story of his life to a group of kids: He too was once a young boy by the name of Little Muck - much like them, but with better manners and a heap of problems. Having lost his father at early age, little Muck is expelled from home by his greedy relatives. He wanders off into the desert hoping to find the merchant who sells good fortune. Amidst the dunes of sand he comes across a small house owned by a wicked woman and her many cats. She wants to make Little Muck her servant, but he manages to escape by stealing a pair of magic shoes which enable him to run faster than any man in the country. From there he heads right into the next set of challenges... Based on a fai

In [30]:
movie_list[12665]

{'Actors': 'Michael Madsen, Maureen McCarthy, Cecil Moe, Rex Flores',
 'Awards': 'N/A',
 'BoxOffice': 'N/A',
 'Country': 'USA',
 'DVD': '20 Sep 1994',
 'Director': 'Edward T. McDougal',
 'Genre': 'Biography, Drama',
 'Language': 'English',
 'Metascore': 'N/A',
 'Plot': 'A man struggling with alcoholism turns to a reverend for help',
 'Poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMTgwNTc3OTQ0Nl5BMl5BanBnXkFtZTcwMDI0OTUyMQ@@._V1_SX300.jpg',
 'Production': 'N/A',
 'Rated': 'N/A',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '3.7/10'}],
 'Released': 'N/A',
 'Response': 'True',
 'Runtime': '90 min',
 'Title': 'Against All Hope',
 'Type': 'movie',
 'Website': 'N/A',
 'Writer': 'Edward T. McDougal (screenplay), Cecil Moe (screenplay), Jean Moe (screenplay), Jerry Newcombe (screenplay)',
 'Year': '1982',
 'imdbID': 'tt0235166',
 'imdbRating': '3.7',
 'imdbVotes': '132',
 'imdb_id': 'tt0235166',
 'keywords': []}

In [33]:
old_movie_list = movie_list[:12666]
len(old_movie_list)

12666

In [39]:
len(old_movie_list)

12666

In [40]:
new_movie_list = old_movie_list + movie_list_keyword
print(len(new_movie_list))

13716


In [41]:
with open("movie_with_keyword.json", 'w') as fout:
    for row in new_movie_list:
        print(json.dumps(row), file=fout)

In [44]:
final_list_keyword = []

for row in new_movie_list:
    if row['keywords']:
        final_list_keyword.append(row)
    else:
        print('keywords is empty')

keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords i

keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords is empty
keywords i

In [45]:
len(final_list)

11630

In [46]:
len(final_list_keyword)

11664

In [47]:
final_list_plot = []

with open("movie_dict_3.json", "r")as fin:
    for line in fin:
        movie_json = json.loads(line)
        if movie_json['Plot'] and movie_json['Plot'] != "N/A":
            final_list_plot.append(movie_json)
        else:
            print('plot is empty')

plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot is empty
plot i

In [48]:
len(final_list_plot)

14647

In [49]:
with open("movie_with_keyword.json", 'w') as fout:
    for row in final_list_keyword:
        print(json.dumps(row), file=fout)

with open("movie_with_plot.json", 'w') as fout:
    for row in final_list_plot:
        print(json.dumps(row), file=fout)